<a href="https://colab.research.google.com/github/khaichiong/meco7312/blob/master/L14_OLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.sandbox.regression.predstd import wls_prediction_std


# Generate data

In [2]:
n = 1000
e = np.random.normal(0, 1, n)
x1 = np.random.exponential(0.5, n)
x2 = np.random.normal(-1, 1, n)
y = 2 - 3*x1 + 0.5*x2 + e

# OLS estimates

In [3]:
X = np.column_stack((np.ones(n), x1, x2))
b = np.linalg.inv(X.T @ X) @ X.T @ y
print(b)

[ 1.97162826 -3.01973911  0.48745232]


# Variance covariance matrix

In [ ]:
np.linalg.inv(X.T @ X)

## Standard errors of coefficients

In [4]:
np.sqrt(np.diag(np.linalg.inv(X.T @ X)))

array([0.05420703, 0.06112069, 0.03175608])

# Using built-in package

In [5]:
X_sm = sm.add_constant(np.column_stack((x1, x2)))
model = sm.OLS(y, X_sm)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1383.
Date:                Wed, 06 Dec 2023   Prob (F-statistic):          2.45e-288
Time:                        17:42:52   Log-Likelihood:                -1393.4
No. Observations:                1000   AIC:                             2793.
Df Residuals:                     997   BIC:                             2807.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9716      0.053     37.258      0.0

# Check Variance with Collinearity and Sample Size

In [ ]:
#varying n and correlation between x1 and x2
n = 100
e = np.random.normal(0, 1, n)
x1 = np.random.exponential(0.5, n)
x2 = x1 + np.random.normal(-1, 0.1, n)
y = 2 - 3*x1 + 0.5*x2 + e
X = np.column_stack((np.ones(n), x1, x2))
np.sqrt(np.diag(np.linalg.inv(X.T @ X)))


# Estimate Variance of Residuals

In [6]:
residuals = y - X @ b
shat = (residuals.T @ residuals) / (n - 3) #(sum of square of residuals)/N-K
np.sqrt(np.diag(shat * np.linalg.inv(X.T @ X)))


array([0.05291879, 0.05966815, 0.0310014 ])

# Heteroskedastic Errors and Robust Standard Errors

In [14]:
# Generation of heteroskedastic errors
n = 1000
x1 = np.random.exponential(0.5, n)
x2 = np.random.normal(-1, 1, n)
e = np.random.normal(0, np.sqrt(np.abs(2*x1 + x2)), n) #heteroskedastic errors
y = 2 - 3*x1 + 0.5*x2 + e

# OLS estimation
X = np.column_stack((np.ones(n), x1, x2))
b = np.linalg.inv(X.T @ X) @ X.T @ y



In [16]:
# Fitted residuals
ehat = y - X @ b

# White's heteroskedastic-robust standard errors
E = np.diag(ehat**2)
white = np.linalg.inv(X.T @ X) @ X.T @ E @ X @ np.linalg.inv(X.T @ X)
robust_se = np.sqrt(np.diag(white))
print(robust_se)


[0.04955327 0.09609369 0.04641988]


## Comparison to non-robust standard errors

In [17]:
shat = (ehat.T @ ehat) / (n - 3)
non_robust_se = np.sqrt(np.diag(shat * np.linalg.inv(X.T @ X)))
print(non_robust_se)


[0.05694311 0.06876623 0.03338701]


## Comparison using standard package

In [21]:
model = sm.OLS(y, X)
results = model.fit(cov_type='HC0')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     798.8
Date:                Wed, 06 Dec 2023   Prob (F-statistic):          8.56e-208
Time:                        18:21:39   Log-Likelihood:                -1455.9
No. Observations:                1000   AIC:                             2918.
Df Residuals:                     997   BIC:                             2933.
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9562      0.050     39.476      0.0

In [22]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     999.3
Date:                Wed, 06 Dec 2023   Prob (F-statistic):          6.58e-239
Time:                        18:21:54   Log-Likelihood:                -1455.9
No. Observations:                1000   AIC:                             2918.
Df Residuals:                     997   BIC:                             2933.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9562      0.057     34.353      0.0

In [23]:
model = sm.OLS(y, X)
results = model.fit()
_, pval, _, f_pval = het_breuschpagan(results.resid, results.model.exog)
pval, f_pval

(2.971579062457799e-08, 2.3036613286707456e-08)